#### Face detection
- 카카오 얼굴인식 관련 리서치 글 : https://tech.kakaoenterprise.com/63
- 네이버 얼굴검출 관련 오픈소스 : https://github.com/clovaai/EXTD_Pytorch

#### 빠르게 하기위한 방법
- sliding window 를 버려야 빨라진다. 2-stage 방식의 detection은 좋은 대안이 못된다.

- 병렬화가 가능해야 합니다.

- CNN 은 gpu 에서 병렬화가 가능하겠지만, 핸드폰에서도 병렬화가 가능한가?
- Apple 은 coreml 이라는 라이브러리를 지원함
  - https://developer.apple.com/documentation/coreml
  - http://machinethink.net/blog/ios-11-machine-learning-for-everyone/
  - 사례 : 16core 뉴럴엔진을 넣은 아이폰12 iPhone 12 Pro 및 iPhone 12 Pro Max

- 스마트폰 제조사가 병렬화를 지원하지 않는다면?
  - ML kit : https://www.slideshare.net/inureyes/ml-kit-machine-learning-sdk
  - tflite : https://www.tensorflow.org/lite?hl=ko
  - tflite 가 안된다면..?
  - 직접 병렬프로그래밍으로 pytorch, tensorflow 같은 툴을 제작 (난이도 ★★★★★)  
    SIMD : https://stonzeteam.github.io/SIMD-병렬-프로그래밍/  
    OpenCL : https://www.khronos.org/opencl/  
    OpenGL ES : https://developer.android.com/guide/topics/graphics/opengl?hl=ko  
 

### Single Stage Object Detection

- object detection 모델을 자세히 설명 : [Object Detection Part 4: Fast Detection Models](https://lilianweng.github.io/lil-log/2018/12/27/object-detection-part-4.html)

- single shot object detectors : [What do we learn from single shot object detectors (SSD, YOLOv3), FPN & Focal loss (RetinaNet)?](https://jonathan-hui.medium.com/what-do-we-learn-from-single-shot-object-detectors-ssd-yolo-fpn-focal-loss-3888677c5f4d)

- 위 글의 번역본 : https://murra.tistory.com/17



### YOLO

#### RCNN과 YOLO의 차이점
![yolo](https://aiffelstaticprd.blob.core.windows.net/media/images/GC-10-L-04.max-800x600.png)  
- RCNN 계열의 가정 :
  "물체가 존재할 것 같은 곳을 backbone network 로 표현할 수 있다." → region proposal network

- YOLO v1 의 가정 :
  "이미지 내의 작은 영역을 나누면 그 곳에 물체가 있을 수 있다." → grid 내에 물체가 존재한다.

- backbone 을 통과한 7x7 feature map 에서 1px 가 1개의 grid 를 의미함 Q. 원본 이미지에서 1개의 grid box 의 사이즈를 구하시오 A. 448 / 7 = 64 ⇒ 64x64

- 7x7 feature map 에서 1) Bounding box 와 관련된 Bbox 개수 x (x, y, w, h, confidence) 5 개 값 2) Class 확률 C 개의 tensor 를 출력

- 최종 출력 개수는 7x7x(5xB + C) 가 된다.



#### YOLO의 grid cell
![yolo2](https://aiffelstaticprd.blob.core.windows.net/media/images/GC-10-L-05.max-800x600.png)  
- Q. 1개의 grid 당 2개의 Bbox 와 20개 클래스를 예측하는 YOLO 를 만들고 싶은 경우 output tensor 의 flatten 했을 때 크기는? 
- A. 7 x 7 x (5 x 2 + 20) = (1470,)


![yolo3](https://aiffelstaticprd.blob.core.windows.net/media/images/GC-10-L-06.max-800x600.png)  
- grid cell 에 속하는 물체를 검출할 책임이 있다
- 1개 grid 에 귀속된 bbox 정보 (x,y,w,h) 의 학습 목표는 bbox gt 와 최대한 동일하게 IoU 를 사용하여 학습되어야 한다.

#### YOLO 의 특징
![yolo4](https://aiffelstaticprd.blob.core.windows.net/media/images/GC-10-L-07.max-800x600.png)  
- 기존 R-CNN 계열 방법은 검출속도가 느림
- Faster R-CNN은 RPN 후보군을 뽑고 localization, classification 을 수행함 이 때 RPN 에서 300개 영역을 제안하는데, objectness 의 숫자가 많을 수록 느려지며 7x7= 49 grid 와 비교해서 봐야한다.


#### YOLO의 Inference 과정
![yolo5](https://aiffelstaticprd.blob.core.windows.net/media/images/GC-10-L-10.max-800x600.png)  
- 7x7 grid 마지막 layer 의 해석
- 7x7 x (30) 에서 30 = 5(x,y,w,h,c) + 5 + 20 (class) 로 이루어진다.
- classification 은 이미 우리가 익히 알고 있는 P( real | pred ) 인 likelihood 를 사용
- confidence score 를 loss 로 만들 때는 P(class | object) * P(object) * IoU 로 표현
- 7x7x2 개의 class confidence score 가 계산


#### YOLO v1 의 loss 함수
![yolo6](https://aiffelstaticprd.blob.core.windows.net/media/images/GC-10-L-11.max-800x600.png)  
![yolo7](https://aiffelstaticprd.blob.core.windows.net/media/images/GC-10-L-12.max-800x600.png)

#### YOLO v1 의 단점
- 각각 grid cell 이 하나의 클래스만 예측 가능하므로 작은 object 에 대해 예측이 어려움
- bbox 형태가 training data 를 통해 학습 → bbox 분산이 너무 넓어 새로운 형태의 bbox 예측이 잘 안 됨.
- 모델 구조상 backbone 만 거친 feature map 대상으로 bbox 정보를 예측하기 때문에 localization 이 다소 부정확

#### YOLO v3
- [Yolo v3의 원리](https://taeu.github.io/paper/deeplearning-paper-yolov3/)

###  SSD 

####  SSD의 특징
#### SSD : Single Shot MultiBox Detector  
--- 
- YOLO 가 1 stage 로 object detection 이 가능해진다는 증명을 한 뒤 1 stage detector 가 수 많은 발전을 이루기 시작
- SSD 는 YOLO v1 에서 grid 를 사용해서 생기는 단점을 해결할 수 있는 몇가지 테크닉을 제안
- image pyramid
- pre-defined anchor box

#### Image Pyramid
---
![ssd](https://aiffelstaticprd.blob.core.windows.net/media/images/GC-10-L-30.max-800x600.png)  
- ImageNet 으로 pretrained 된 VGG16 을 사용
- VGG 에서 pooling 을 거친 block 은 하나의 image feature 로 사용가능
  - YOLO 에서 7x7 feature map 하나만을 사용했다면,
  - SSD 는 38x38, 19x19, 10x10, 5x5, 3x3 … 을 사용함
- 각 feature map 은 YOLO 에 관점에서 보면 원본 이미지에서 grid 크기를 다르게 하는 효과가 있음
  - 5x5 feature map 에서 grid 가 너무 커서 small object 를 못찾는 문제를 38x38 feature map 에서 찾을 수 있는 단서를 마련

#### Workflow
--- 
- YOLO v1 의 두번째 단점은 box 정보 (x,y,w,h) 를 예측하기 위한 seed 정보가 없기 때문에 넓은 bbox 분포를 모두 학습할 수 없다는 점 → 이로 인한 성능 손실이 존재할 수 있다.
- faster rcnn 등 에서 사용하는 anchor 를 적용할 필요가 있다. ex) (가정) 개가 등장하는 bounding box 만의 x, y, w, h 특성이 존재할 것 → pre-defined 된 box 부터 x,y,w,h 를 refinement 하는 layer 를 추가하는 것이 이득이다. 이 anchor box 를 SSD 에서는 default box 라고 부름  
![workflow](https://aiffelstaticprd.blob.core.windows.net/media/images/GC-10-L-31.max-800x600.png)  
- SSD 의 framework  
  (a) : 이미지, GT 데이터셋  
  (b) : (vgg backbone 에 가까운) fine-grained feature map. 8x8 grid 에서 각각의 grid 에 3개 anchor box 를 적용할 수 있음.
  고양이는 크기가 작기 때문에 (a) 의 고양이는 8x8 feature map 내 grid 중 1개의 anchor box 로 부터 학습될 수 있음  
  (c) : 개의 경우 크고 세로로 긴 경향을 보이기 때문에 receptive field 가 넓은 4x4 feature map 이 사용됨  
    
- Default box를 위한 scale. 여러 크기의 default box 생성을 위해 아래와 같은 식을 사용
![db](https://aiffelstaticprd.blob.core.windows.net/media/original_images/GC-10-L-32.png)  
![db2](https://aiffelstaticprd.blob.core.windows.net/media/images/GC-10-L-33.max-800x600.png)  
![db3](https://aiffelstaticprd.blob.core.windows.net/media/images/GC-10-L-34.max-800x600.png)  
![db4](https://aiffelstaticprd.blob.core.windows.net/media/images/GC-10-L-35.max-800x600.png)

#### SSD Loss function
--- 
![lf](https://aiffelstaticprd.blob.core.windows.net/media/images/GC-10-L-36.max-800x600.png)
![lf2](https://aiffelstaticprd.blob.core.windows.net/media/images/GC-10-L-37.max-800x600.png)
![lf3](https://aiffelstaticprd.blob.core.windows.net/media/images/GC-10-L-38.max-800x600.png)
![lf4](https://aiffelstaticprd.blob.core.windows.net/media/images/GC-10-L-39.max-800x600.png)
![lf5](https://aiffelstaticprd.blob.core.windows.net/media/images/GC-10-L-40.max-800x600.png)
![lf6](https://aiffelstaticprd.blob.core.windows.net/media/images/GC-10-L-41.max-800x600.png)
![lf7](https://aiffelstaticprd.blob.core.windows.net/media/images/GC-10-L-42.max-800x600.png)

#### Hard negative mining
---
![hnm](https://aiffelstaticprd.blob.core.windows.net/media/images/GC-10-L-43.max-800x600.png)

#### SSD의 성능
---
![ssd2](https://aiffelstaticprd.blob.core.windows.net/media/images/GC-10-L-45.max-800x600.png)

### Face Detection을 위한 모델
- [S3FD](https://seongkyun.github.io/papers/2019/03/21/S3FD/)
- [S3FD](https://arxiv.org/abs/1708.05237)
- [DSFD](https://arxiv.org/pdf/1810.10220.pdf)
- [RetinaFace](https://arxiv.org/pdf/1905.00641.pdf)